In [4]:
import json
import pandas as pd
import re

In [55]:
with open('data/deepseek_8bit_finetuned.json', 'r') as f:
    output = json.load(f)

In [56]:
def fix_single_quotes(json_str):
    return json_str.replace("'", '"')

In [61]:
for key in output:
    paper_data = output[key]
    try:
        think_tag = '</think>'
        if think_tag in paper_data:
            paper_data = paper_data.split(think_tag)[-1]
        json_match = re.search(r"\{.*\}", paper_data, re.DOTALL)
        raw_json = json_match.group(0).strip()
        fixed_json = fix_single_quotes(raw_json)
        # print(key)
        # print(fixed_json)
        output[key] = fixed_json
    except Exception as e:
        print(key)
        print(paper_data)
        print(e)
        # print(paper_data)
json_string = json.dumps(output, indent=4)

47
None
argument of type 'NoneType' is not iterable
24
None
argument of type 'NoneType' is not iterable
4
None
argument of type 'NoneType' is not iterable
103
None
argument of type 'NoneType' is not iterable
68
None
argument of type 'NoneType' is not iterable
0
None
argument of type 'NoneType' is not iterable


In [65]:
json_string

'{\n    "115": "{\\"perovskite_composition\\": None, \\"electron_transport_layer\\": None, \\"hole_transport_layer\\": None, \\"structure\\": \\"PIN\\", \\"test_1\\": {\\"passivating_molecule\\": \\"anisole\\", \\"stability_type\\": \\"ISOSD\\", \\"temperature\\": \\"85\\", \\"humidity\\": None, \\"time\\": \\"1000\\", \\"control_pce\\": None, \\"treated_pce\\": \\"23.15\\", \\"control_voc\\": None, \\"treated_voc\\": \\"1.155\\", \\"efficiency_cont\\": \\"90\\", \\"efficiency_tret\\": \\"95\\"}}",\n    "112": "{\\"perovskite_composition\\": \\"MA0.7FA0.3PbI3\\", \\"electron_transport_layer\\": \\"C60\\", \\"pin_nip_structure\\": \\"PIN\\", \\"hole_transport_layer\\": \\"poly[bis(4-phenyl)(2,4,6-trimethylphenyl)amine]\\", \\"test_1\\": {\\"test_name\\": \\"ISOSLT\\", \\"temperature\\": \\"65\\", \\"humidity\\": \\"60 \\u00b1 10\\", \\"time\\": \\"1000\\", \\"passivating_molecule\\": \\"benzylhydrazine chloride\\", \\"control_pce\\": None, \\"treated_pce\\": \\"23.2\\", \\"control_voc\\

In [68]:
with open('data/deepseek_8bit_finetuned.json', 'w') as f:    
    json.dump(output, f, indent=4)

In [67]:
with open('data/deepseek_8bit_finetuned_2.json', 'r') as f:
    result = json.load(f)

result

{'115': '{"perovskite_composition": None, "electron_transport_layer": None, "hole_transport_layer": None, "structure": "PIN", "test_1": {"passivating_molecule": "anisole", "stability_type": "ISOSD", "temperature": "85", "humidity": None, "time": "1000", "control_pce": None, "treated_pce": "23.15", "control_voc": None, "treated_voc": "1.155", "efficiency_cont": "90", "efficiency_tret": "95"}}',
 '112': '{"perovskite_composition": "MA0.7FA0.3PbI3", "electron_transport_layer": "C60", "pin_nip_structure": "PIN", "hole_transport_layer": "poly[bis(4-phenyl)(2,4,6-trimethylphenyl)amine]", "test_1": {"test_name": "ISOSLT", "temperature": "65", "humidity": "60 ± 10", "time": "1000", "passivating_molecule": "benzylhydrazine chloride", "control_pce": None, "treated_pce": "23.2", "control_voc": None, "treated_voc": "1.17", "stability_test_description": "Operational stability test of encapsulated PSCs under one sun equivalent illumination in air (neither cooling nor UV filters were used in this tes

In [3]:
df = pd.read_csv('data/llama3b_output_fully_nested.csv')
df

,Unnamed: 0.1,Unnamed: 0,id,full_text,filtered_text,json_output
0,0,0,115,Paper #: 115\nLiquid medium annealing for fabr...,Liquid medium annealing for fabricating durabl...,"{'perovskite_composition': None, 'electron_tra..."
1,1,1,112,Paper #: 112\nIodine reduction for reproducibl...,Iodine reduction for reproducible and high-per...,"{'perovskite_composition': 'MA0.7FA0.3PbI3', '..."
2,2,2,72,Paper #: 72\nSuppressed Ion Migration in Reduc...,T oday's perovskite solar cells (PSCs) require...,{'perovskite_composition': 'Cs0.05FA0.8MA0.15P...
3,3,3,75,Paper #: 75\nDynamical Evolution of the 2D/3D ...,UV-VIS Absorption The absorption spectra of t...,"{'perovskite_composition': None, 'electron_tra..."
4,4,4,40,Paper #: 40\nDamp heat-stable perovskite solar...,Damp heat-stable perovskite solar cells with t...,"{'perovskite_composition': 'FAMAPbI3', 'electr..."
...,...,...,...,...,...,...
144,144,144,92,Paper #: 92\n\nBanners \n Banners New for 2018...,Electronic Table of Contents (eToC) \n Standa...,"{'perovskite_composition': None, 'electron_tra..."
145,145,145,123,Paper #: 123\nConstructing heterojunctions by ...,P erovskite solar cells (PSCs) have reached po...,"{'perovskite_composition': 'FAPbI3', 'electron..."
146,146,146,31,Paper #: 31\n\nWe performed 1 H-NMR to support...,We performed 1 H-NMR to support our proposed r...,{'perovskite_composition': 'FA0.6MA0.4Pb0.6I3'...
147,147,147,36,Paper #: 36\nUnderstanding Performance Limitin...,Understanding Performance Limiting Interfacial...,{'perovskite_composition': 'Cs0.05(FA0.83MA0.1...


In [4]:
df = df[df["json_output"].isna() == False] 

In [41]:
df

,Unnamed: 0.1,Unnamed: 0,id,full_text,filtered_text,json_output
0,0,0,115,Paper #: 115\nLiquid medium annealing for fabr...,Liquid medium annealing for fabricating durabl...,"{'perovskite_composition': None, 'electron_tra..."
1,1,1,112,Paper #: 112\nIodine reduction for reproducibl...,Iodine reduction for reproducible and high-per...,"{'perovskite_composition': 'MA0.7FA0.3PbI3', '..."
2,2,2,72,Paper #: 72\nSuppressed Ion Migration in Reduc...,T oday's perovskite solar cells (PSCs) require...,{'perovskite_composition': 'Cs0.05FA0.8MA0.15P...
3,3,3,75,Paper #: 75\nDynamical Evolution of the 2D/3D ...,UV-VIS Absorption The absorption spectra of t...,"{'perovskite_composition': '2D/3D perovskite',..."
5,5,5,91,Paper #: 91\nMethylammonium Chloride Induces I...,Electronic Table of Contents (eToC) \n Email ...,"{'perovskite_composition': 'FAPbI3', 'electron..."
...,...,...,...,...,...,...
144,144,144,92,Paper #: 92\n\nBanners \n Banners New for 2018...,Electronic Table of Contents (eToC) \n Standa...,"{'perovskite_composition': None, 'electron_tra..."
145,145,145,123,Paper #: 123\nConstructing heterojunctions by ...,P erovskite solar cells (PSCs) have reached po...,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
146,146,146,31,Paper #: 31\n\nWe performed 1 H-NMR to support...,We performed 1 H-NMR to support our proposed r...,{'perovskite_composition': 'FA0.6MA0.4Sn0.6Pb0...
147,147,147,36,Paper #: 36\nUnderstanding Performance Limitin...,Understanding Performance Limiting Interfacial...,{'perovskite_composition': 'Cs0.05FA0.83MA0.17...


In [7]:
import re
output = {}
i = 0
for index, row in df.iterrows():
    i += 1
    json_match = re.search(r"\{.*\}", row["json_output"], re.DOTALL)
    if json_match:
        raw_json = json_match.group(0).strip()
    else:
        raw_json = None
    output[str(row["id"])] = raw_json

In [8]:
output

{'115': "{'perovskite_composition': None, 'electron_transport_layer': None, 'pin_nip_structure': None, 'hole_transport_layer': None, 'test_1': {'test_name': 'ISOS-L', 'temperature': '85', 'time': '1000', 'humidity': '30-50', 'passivating_molecule': 'anisole', 'control_pce': '22.7', 'control_voc': '1.16', 'treated_pce': '23.1', 'treated_voc': '1.17'}}",
 '112': "{'perovskite_composition': 'MA0.7FA0.3PbI3', 'electron_transport_layer': 'C60', 'pin_nip_structure': 'NIP', 'hole_transport_layer': 'Spiro-OMeTAD', 'test_1': {'passivating_molecule': 'Benzylhydrazine Hydrochloride', 'humidity': None, 'temperature': '65', 'time': '1000', 'retained_percentage_cont': None, 'retained_percentage_tret': '90', 'control_pce': '21.8', 'treated_pce': '23.4', 'control_voc': '1.14', 'treated_voc': '1.17', 'test_name': 'ISOSLT', 'device_type': 'module', 'efficiency': '18.5', 'area': '35.8', 'certified': True}}",
 '72': "{'perovskite_composition': 'Cs0.05FA0.8MA0.15PbI2.88Br0.04Cl0.08', 'electron_transport_la

In [9]:
with open('data/llama_3b_8bit_fully_nested.json', 'w') as f:
    json.dump(output, f)

In [44]:
df = pd.read_json('data/deepseek_8bit_finetuned.json')
df

ValueError: If using all scalar values, you must pass an index